# Inference and Scoring

* 표준어 > 경상도 예시입니다.
* 예시로 사용한 모델은 1 epoch만 학습한 모델입니다.

In [1]:
import pandas as pd
import torch
from transformers import BartForConditionalGeneration
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
from tqdm import tqdm
from torchtext.data.metrics import bleu_score

## load tokenizer & model

In [2]:
tokenizer = get_kobart_tokenizer()

using cached model. /home/eunjin/dialect_convert/projects/.cache/kobart_base_tokenizer_cased_cf74400bce.zip


In [3]:
model = BartForConditionalGeneration.from_pretrained('model_results/s2d/kyeongsang/model')

In [4]:
model.eval()
model.to('cuda')
print('>> model set')

>> model set


## load test data

In [5]:
test_df=pd.read_csv('data/kyeongsang/test_cleaned.tsv',sep='\t')

In [6]:
test_df.sample(3)

,standard,dialect,mp_d,mp_s,mp_idx
4820,잘 시켜서 먹더라 어제도 스치로폼 한 박스가 와서 뜯어 보니깐,잘 시키가 묵드라 어제도 스치로폼 한 박스가 와서 뜯어 보니깐,시키가_묵드라,시켜서_먹더라,2 3
10928,애들은 편하게 있고 그런 식으로 해가지고 오히려 조금,애들은 편하게 있고 그런 식으로 해가지고 오히려 쫌,쫌,조금,8
11301,내가 대신 교수님한테 제출하고 이렇게 했단 말이에요 그래서,내가 대신 교수님한테 제출하고 이켔단 말이에요 그래서,이켔단,이렇게 했단,5


### inference test

In [7]:
idx = 6013
sent = test_df['standard'][idx]
print('input: ' , sent)
print('gold: ' , test_df['dialect'][idx])
# prepare input
inputs=tokenizer(sent,return_tensors='pt')
# max_length, num_beams are hypperparameters
outputs=model.generate(inputs['input_ids'].to('cuda'), eos_token_id=1, max_length=128, num_beams=5)
print('generation: ', tokenizer.decode(outputs[0]))

input:  토익을 한 번 쳤었는데 한 번 아니고 두 번 쳤었지.
gold:  토익을 한 번 쳤었는데 한 번 아이고 두 번 쳤었지.
generation:  <usr> 토익을 한 번 쳤었는디 한 번 아니고 두 번 쳤었지.</s>


### scoring

In [8]:
preds=[]
for sent in tqdm(test_df['standard'][:100]):
    inputs=tokenizer(sent,return_tensors='pt')
    outputs=model.generate(inputs['input_ids'].to('cuda'), eos_token_id=1, max_length=128, num_beams=5)
    preds.append(tokenizer.decode(outputs[0][1:-1])) # remove special tokens

100%|██████████| 100/100 [00:22<00:00,  4.49it/s]


In [9]:
preds = [p.split() for p in preds]
targets = [[d.split()] for d in test_df['dialect'][:100]]

In [10]:
bleu_score(preds, targets)

0.8944592475891113